In [1]:
import folium
from folium import plugins
import ipywidgets
import geocoder
import geopy
import numpy as np
import pandas as pd
import json
import csv
from folium.plugins import Search

In [2]:
RadiationStart = pd.read_csv("workingOn_-_Copy.csv")
RadiationStart = RadiationStart.iloc[:,1:]
RadiationStart = RadiationStart.iloc[0:3111]
for i in range(len(RadiationStart)):
    RadiationStart.iloc[i,0] = RadiationStart.iloc[i,0].replace(' County','')

In [3]:
RadiationStart['Avg Daily Precipitation (mm)'] = pd.to_numeric(RadiationStart['Avg Daily Precipitation (mm)'])
RadiationStart['Avg Day Land Surface Temperature (F)'] = pd.to_numeric(RadiationStart['Avg Day Land Surface Temperature (F)'])
RadiationStart['Avg Daily Sunlight (KJ/m2)'] = pd.to_numeric(RadiationStart['Avg Daily Sunlight (KJ/m2)'])

In [4]:
tempNorm = (RadiationStart['Avg Day Land Surface Temperature (F)']-72)/np.std(RadiationStart['Avg Day Land Surface Temperature (F)'])
tempNorm = -np.abs(tempNorm)
tempNorm = tempNorm-np.mean(tempNorm)
rainNorm = (RadiationStart['Avg Daily Precipitation (mm)']-np.mean(RadiationStart['Avg Daily Precipitation (mm)']))/np.std(RadiationStart['Avg Daily Precipitation (mm)'])
sunNorm = (RadiationStart['Avg Daily Sunlight (KJ/m2)']-np.mean(RadiationStart['Avg Daily Sunlight (KJ/m2)']))/np.std(RadiationStart['Avg Daily Sunlight (KJ/m2)'])
bois = [tempNorm,rainNorm,sunNorm]

In [5]:
coefs = [3,-3,10]
RadiationStart['Ratio'] = 0
for i in range(len(coefs)):
    RadiationStart['Ratio'] = RadiationStart['Ratio'] + coefs[i]*bois[i]
RadiationStart['Ratio'] = RadiationStart['Ratio']  - np.min(RadiationStart['Ratio'] )
RadiationStart['Ratio'] = RadiationStart['Ratio'] *(10/np.max(RadiationStart['Ratio'] ))

In [6]:
counties = []
for i in RadiationStart['County']:
    counties.append(i)

In [7]:
url = (
      "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
  )
tester = json.load(open("us-counties.json"))
temp = []
for i in range(len(tester['features'])):
    temp.append(tester['features'][i]['properties']['name'])
display = RadiationStart[['County','Avg Daily Precipitation (mm)','Avg Daily Sunlight (KJ/m2)',
                          'Avg Day Land Surface Temperature (F)','Ratio']]
m = folium.Map(location=[39.8283, -98.5795], zoom_start=4.4, control_scale=True)

cp = folium.Choropleth(
      geo_data=tester,
      name="choropleth",
      data=display,
      columns=["County",'Ratio'] ,
      key_on="feature.properties.name",
      fill_color='Reds',
      nan_fill_color='white',
      fill_opacity=0.7,
      line_opacity=0.2,
      legend_name="Solar Usefulness",
  ).add_to(m)

county_data_indexed = display.set_index('County')

for s in cp.geojson.data['features']:
    if  s['properties']['name'] in counties:
        s['properties']['Ratio'] = np.around(np.mean(pd.to_numeric(county_data_indexed.loc[s['properties']['name'], 'Ratio'])),2)
        s['properties']['Avg Daily Precipitation (mm)'] = np.mean(pd.to_numeric(county_data_indexed.loc[s['properties']['name'], 'Avg Daily Precipitation (mm)']))
        s['properties']['Avg Daily Sunlight (KJ/m2)'] = np.mean(pd.to_numeric(county_data_indexed.loc[s['properties']['name'], 'Avg Daily Sunlight (KJ/m2)']))
        s['properties']['Avg Day Land Surface Temperature (F)'] = np.mean(pd.to_numeric(county_data_indexed.loc[s['properties']['name'], 'Avg Day Land Surface Temperature (F)']))
    else:
        s['properties']['Ratio'] = "N/A"

folium.GeoJsonTooltip(["name", "Ratio"]).add_to(cp.geojson)

draw = plugins.Draw(
    export=True,
    filename=tester,
    draw_options={
          'polyline':False,
          'rectangle':False,
          'polygon':True,
          'circle':False,
          'marker':True,
          'circlemarker':False},
    )
draw.add_to(m)

folium.LayerControl().add_to(m)

folium.plugins.Geocoder(collapsed=False,position='topleft',add_marker=True).add_to(m)

m.save("Output.html") 
m